In [ ]:
import pretty_midi
import os
import scipy.io.wavfile
import time
import IPython.display as ipd 
import numpy as np
import sklearn.cluster
import supervised_pca
import matplotlib.pyplot as plt

In [ ]:
# generate the individual instrument wavs for different pitches
note_wavs_dir = "/home/faraaz/workspace/music-transcription/data/note_wavs/"
sf2_path = "/usr/share/sounds/sf2/FluidR3_GM.sf2"

num_programs = 16
min_pitch = 30
max_pitch = 100
note_length = 2
velocity = 80
sample_rate = 44100
start = time.time()

for program in range(num_programs):
    break
    program = program * 8 + 2
    for pitch in range(min_pitch, max_pitch+1):
        note = pretty_midi.Note(velocity, pitch, 0, note_length)
        instrument  = pretty_midi.Instrument(program)
        instrument.notes = [note]
        pm = pretty_midi.PrettyMIDI()
        pm.instruments = [instrument]
        
        note_samples = pm.fluidsynth(fs=sample_rate, sf2_path=sf2_path)
        filename = os.path.join(note_wavs_dir, "{}_{}.wav".format(program, pitch))
        scipy.io.wavfile.write(filename, sample_rate, note_samples)
    print("{}. {}s".format(program, time.time()-start))
    start = time.time()
        

In [ ]:
note_wav_file = "/home/faraaz/workspace/music-transcription/data/note_wavs/0_50.wav"
_, note_samples = scipy.io.wavfile.read(note_wav_file)
print(note_samples.shape)
ipd.Audio(data=note_samples, rate=sample_rate)

In [ ]:
# load the note embeddings
note_embeds_dir = "/home/faraaz/workspace/music-transcription/data/note_embeds/"
note_embeds = []
program_labels = []
pitch_labels = []

used_programs = set()
for program in range(128):
    for pitch in range(min_pitch, max_pitch+1):
        npy_file = os.path.join(note_embeds_dir, "{}_{}_embeddings.npy".format(program, pitch))
        if not os.path.isfile(npy_file):
            continue
        used_programs.add(program)
        filename = os.path.join(note_embeds_dir, npy_file)
        note_embed = np.load(filename)
        note_embeds.append(note_embed)
        program_labels.append(program)
        pitch_labels.append(pitch)

note_embeds = np.array(note_embeds)
program_labels = np.array(program_labels)
pitch_labels = np.array(pitch_labels)
print(note_embeds.shape)
print(note_embeds[0][0])
print(note_embeds[1][0])
print(len(used_programs))

# filename = os.path.join(note_embeds_dir, "{}_{}_embeddings.npy".format(0, 50))
# print(filename)
# note_embed = np.load(filename)
# print(note_embed.shape)

In [ ]:
X = np.reshape(note_embeds, (note_embeds.shape[0], -1))
print(X[0][:10])
print(X[1][:10])
y = program_labels
print(X.shape)
kmeans = sklearn.cluster.AgglomerativeClustering(n_clusters=8)
note_preds = kmeans.fit_predict(X)
print(note_preds.shape)

In [ ]:
total_notes = len(note_embeds)
note_range = max_pitch - min_pitch
results_matrix = []
for i in range(8):
    results_matrix.append([0]*8)
    for pitch in range(note_range):
        cluster = note_preds[i*8 + pitch]
        results_matrix[i][cluster] += 1

for i in range(8):
    print("{}.\t{}".format(i, results_matrix[i]))

In [ ]:
total_notes = len(note_embeds)
note_range = max_pitch - min_pitch
results_matrix = []
for pitch in range(note_range):
    results_matrix.append([0]*num_programs)
    for i in range(8):
        cluster = note_preds[i*8 + pitch]
        results_matrix[pitch][cluster] += 1

for pitch in range(note_range):
    print("{}.\t{}".format(pitch, results_matrix[pitch]))

In [ ]:
print(note_embeds.shape)
X = np.reshape(note_embeds[:, :63, :], (len(note_embeds), -1))
# X_train = X[len(X)//5:]
# X_test = X[:len(X)//5]
# print(X_train.shape)
# print(X_test.shape)
y = pitch_labels
# y_train = pitch_labels[len(pitch_labels)//5:]
# y_test = pitch_labels[:len(pitch_labels)//5]
print(y.shape)

pca = supervised_pca.SupervisedPCARegressor(normalize=True)
pca.fit(X, y)
print(pca.get_n_components())

In [ ]:
# print(pca.score(X_test, y_test))

coefs = abs(pca.get_coefs())
print(sorted(coefs)[-10:])
ordered_components = np.argsort(coefs)
print(ordered_components[-10:])
print(coefs[ordered_components][-10:])

In [ ]:
transformed_X = pca.get_transformed_data(X)
print(transformed_X.shape)
transformed_X = transformed_X[:, ordered_components]
transformed_X = transformed_X[:, :2]
#transformed_X = np.delete(transformed_X, pitch_component, axis=1)
print(transformed_X.shape)

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters=8)
note_preds = kmeans.fit_predict(transformed_X)
print(note_preds.shape)

In [ ]:
total_notes = len(transformed_X)
note_range = max_pitch - min_pitch
results_matrix = []
for i in range(8):
    results_matrix.append([0]*8)
    for pitch in range(note_range):
        cluster = note_preds[i*8 + pitch]
        results_matrix[i][cluster] += 1

for i in range(8):
    print("{}.\t{}".format(i, results_matrix[i]))

In [ ]:
total_notes = len(transformed_X)
note_range = max_pitch - min_pitch
results_matrix = []
for pitch in range(note_range):
    results_matrix.append([0]*num_programs)
    for i in range(8):
        cluster = note_preds[i*8 + pitch]
        results_matrix[pitch][cluster] += 1

for pitch in range(note_range):
    print("{}.\t{}".format(pitch, results_matrix[pitch]))

In [ ]:
print(transformed_X.shape)
fig, ax = plt.subplots(figsize=(20,20))
plt.axis('equal')
ax.scatter(transformed_X[:,-2], transformed_X[:,-1], c=program_labels)

for i in range(len(transformed_X)):
    ax.annotate(pitch_labels[i], (transformed_X[i,-2], transformed_X[i,-1]))

In [ ]:
print(transformed_X.shape)
fig, ax = plt.subplots()
ax.scatter(transformed_X[:,0], transformed_X[:,1])

for i in range(len(transformed_X)):
    ax.annotate(pitch_labels[i], (transformed_X[i,0], transformed_X[i,1]))

In [ ]:
transformed_X = pca.get_transformed_data(X)
print(transformed_X.shape)
transformed_X = transformed_X[:, ordered_components]
#transformed_X = transformed_X[:, :2]
#transformed_X = np.delete(transformed_X, pitch_component, axis=1)
print(transformed_X.shape)

In [ ]:
# points_dict = {}
# for i in range(len(program_labels)):
#     if tuple(transformed_X[i]) not in points_dict:
#         points_dict[tuple(transformed_X[i])] = set()
#     points_dict[tuple(transformed_X[i])].add(program_labels[i])

# for point in points_dict:
#     print("{}: {}".format("hi", points_dict[point]))